# INITIAL SETUP

In [19]:
from datetime import datetime
import requests, json, os
from typing import Dict, Annotated, Optional, List, Union
from dotenv import load_dotenv
load_dotenv()

# from services.twilio import client
# from services.cache import get_agent_metadata, get_all_agents
# from services.chat.chat import similarity_search, get_embedding
# from services.db.supabase_services import supabase_client
# supabase = supabase_client()

# from app.core.config import settings

from composio import Composio

# Initialize the Composio client
client = Composio()

In [20]:
""" Get all connected apps and accounts """
""" store connection ID to each user_id in supabase """


# # Get the list of integrations (connected apps)
# integrations = client.integrations.get()
# print("Connected Apps:")
# for integration in integrations:
#     print(f"• App: {integration.appName}")
#     print(f"  ID: {integration.id}")

# Get the list of connected accounts (existing connections)
connected_accounts = client.connected_accounts.get()
print("\nExisting Connections:")
for account in connected_accounts:
    print(f"• ID: {account.id}")
    print(f"  App: {account.appName}")
    print(f"  Status: {account.status}")



Existing Connections:
• ID: 78c4b699-96f0-45f5-a90a-4fdf687a1e10
  App: googlecalendar
  Status: ACTIVE
• ID: d5093510-1222-4a7d-b324-c6b3ec812844
  App: slack
  Status: ACTIVE
• ID: 50fa2ff7-b752-4513-88b5-d43b66168ccf
  App: slack
  Status: ACTIVE
• ID: 070110b2-cc6b-4eca-991a-8e88cbad5451
  App: slack
  Status: INITIATED
• ID: 23bf8b45-38ff-417d-b410-f030dfd76bc6
  App: gmail
  Status: INITIATED
• ID: af95f2cb-7bfe-4dd4-a461-adce7fb37560
  App: slack
  Status: INITIATED
• ID: de29a11a-54dc-42a7-81f7-fd51da877eee
  App: slack
  Status: INITIATED
• ID: 143d0a9c-d450-4cdd-88ff-c06f91e3a632
  App: notion
  Status: INITIATED
• ID: 698e11c5-5356-4dbb-85e7-95e0d88eca3e
  App: gmail
  Status: INITIATED
• ID: 1777eca0-46d3-4e35-b43b-a4ee8333dda5
  App: notion
  Status: INITIATED
• ID: e445e237-90ca-4505-8854-03398f3e4445
  App: googlecalendar
  Status: INITIATED
• ID: a9358d5a-7907-4d93-b087-e39b9fd0a3b2
  App: gmail
  Status: INITIATED
• ID: 4016a6e6-a330-4839-888a-87451c938cd0
  App: noti

In [4]:
""" checks if entity already exists """

existing_entity = composio_client.get_entity(id="user_2mmXezcGmjZCf88gT2v2waCBsXv")
existing_connections = existing_entity.get_connections()

if existing_connections:
    print("Entity already exists with connections:", existing_connections)
else:
    print("Entity is newly created or has no connections")


# # Check for connected accounts for the new entity
# try:
#     connected_account = new_entity.get_connection(app="outlook")
#     print(f"Connected Outlook account: {connected_account}")
# except Exception as e:
#     print("No connected Slack account found")

# # Perform actions using the new entity
# actions = new_entity.get_tools(app_name="github")

existing_connections

NameError: name 'composio_client' is not defined

# Apps

### Composio

In [5]:
from openai import OpenAI
import os 
from dotenv import load_dotenv
from pydantic import BaseModel
from composio_openai import ComposioToolSet, App

load_dotenv()

class TaskRequest(BaseModel):
    task: str  # This will contain the natural language task description
b
# Initialize clients
openai_client = OpenAI()
composio_toolset = ComposioToolSet()
tools = composio_toolset.get_tools(apps=[App.PIPEDRIVE])


response = openai_client.chat.completions.create(
    model="gpt-4o-mini",
    tools=tools,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "add a lead to pipedrive"},
    ],
)

result = composio_toolset.handle_tool_calls(response)

result


[2025-01-29 22:46:12,738][INFO] Logging is set to INFO, use `logging_level` argument or `COMPOSIO_LOGGING_LEVEL` change this
/Users/michaelali/Repos/flowon_ai/venv/lib/python3.12/site-packages/composio/client/collections.py:1151: UserWarning: Using all actions of an app is not recommended for production.Learn more: https://docs.composio.dev/patterns/tools/use-tools/use-specific-actions
  warnings.warn(


ComposioSDKError: No connected account found for app `PIPEDRIVE`; Run `composio add pipedrive` to fix this

In [ ]:
import requests
from dataclasses import dataclass
from typing import Optional
import os 
from dotenv import load_dotenv
load_dotenv()

@dataclass
class PipedrivePersonData:
    name: str
    email: Optional[str] = None 
    phone: Optional[str] = None

    def validate(self):
        if not self.name:
            raise ValueError("Name is required")

@dataclass
class PipedriveLeadData:
    title: str
    person_id: Optional[int] = None 
    organization_id: Optional[int] = None

    def validate(self):
        if not self.title:
            raise ValueError("Title is required")
        if self.person_id is None and self.organization_id is None:
            raise ValueError("Either person_id or organization_id must be set")

@dataclass
class PipedriveNoteData:
    content: str
    lead_id: Optional[int] = None
    deal_id: Optional[int] = None
    person_id: Optional[int] = None
    organization_id: Optional[int] = None

    def validate(self):
        if not self.content:
            raise ValueError("Content is required")
        
        # Check if at least one ID is provided
        has_id = any([
            self.lead_id is not None,
            self.deal_id is not None,
            self.person_id is not None,
            self.organization_id is not None
        ])
        if not has_id:
            raise ValueError("At least one of lead_id, deal_id, person_id, or organization_id must be set")


api_token = os.getenv("PIPEDRIVE_API_KEY")

def create_person(api_token: str, lead_data: PipedrivePersonData) -> dict:
    """
    Create a lead in Pipedrive using the API.
    
    Args:
        api_token: Pipedrive API token
        lead_data: PipedriveLeadData object containing lead information
        
    Returns:
        dict: Response from the Pipedrive API
    """
    url = "https://companydomain.pipedrive.com/api/v1/persons"
    
    headers = {
        "x-api-token": api_token,
        "Content-Type": "application/json"
    }

    # Validate the lead data
    lead_data.validate()
    
    # Convert dataclass to dict, excluding None values
    data = {k: v for k, v in lead_data.__dict__.items() if v is not None}
    
    response = requests.post(url, headers=headers, json=data)
    return response.json()

def create_lead(api_token: str, lead_data: PipedriveLeadData) -> dict:
    """
    Create a lead in Pipedrive using the API.
    
    Args:
        api_token: Pipedrive API token
        lead_data: PipedriveLeadData object containing lead information
        
    Returns:
        dict: Response from the Pipedrive API
    """
    url = "https://companydomain.pipedrive.com/api/v1/leads"
    
    headers = {
        "x-api-token": api_token,
        "Content-Type": "application/json"
    }

    # Validate the lead data
    lead_data.validate()
    
    # Convert dataclass to dict, excluding None values
    data = {k: v for k, v in lead_data.__dict__.items() if v is not None}
    
    response = requests.post(url, headers=headers, json=data)
    return response.json()

""" LLM to update after interaction with a lead"""
def add_note(api_token: str, note_data: PipedriveNoteData) -> dict:
    """
    Add a note in Pipedrive using the API.
    
    Args:
        api_token: Pipedrive API token
        note_data: PipedriveNoteData object containing note information
        
    Returns:
        dict: Response from the Pipedrive API
    """
    url = "https://companydomain.pipedrive.com/api/v1/notes"
    
    headers = {
        "x-api-token": api_token,
        "Content-Type": "application/json"
    }

    # Validate the note data
    note_data.validate()
    
    # Convert dataclass to dict, excluding None values
    data = {k: v for k, v in note_data.__dict__.items() if v is not None}
    
    response = requests.post(url, headers=headers, json=data)
    return response.json()

def get_leads(api_token: str) -> dict:
    url = "https://companydomain.pipedrive.com/api/v1/leads"
    headers = {
        "x-api-token": api_token,
        "Content-Type": "application/json"
    }
    response = requests.get(url, headers=headers)
    return response.json()

add_note(api_token)

# ## LLM will request this data.
# person_data = PipedrivePersonData(
#     name="big boi",
#     email="bigboi@gmail.com",
#     phone="1234567890"
# )

# person_response = create_person(api_token, person_data)
# print(person_response)


# lead_data = PipedriveLeadData(
#     title="big boi",
#     person_id=person_response['data']['id']  # Or could use organization_id instead
# )

# lead_response = create_lead(api_token, lead_data)
# print(lead_response)

### Outlook

In [24]:
""" this function will be triggered in endpoint /composion/new_connection """
""" then to store the connection ID to the user_id in supabase """
""" frontend to have button to "generate auth link", response from this endpoint will be the redirect url """
### entity ID will be created for each user. 
""" {"app": "outlook", "connectedAccountId" : "xxx", "client_id" : "xxx" } """

"""Auth"""
from composio import ComposioToolSet, App
from composio import Composio
from composio_openai import ComposioToolSet, Action
from openai import OpenAI

# Initialize the Composio client
composio_client = Composio()

# # Create a new entity with a custom ID
# new_entity = composio_client.get_entity(id="user_2mmXezcGmjZCf88gT2v2waCBsXv")

toolset = ComposioToolSet()
entity = toolset.get_entity(id="user_2mmXezcGmjZCf88gT2v2waCBsXvww")

# request = entity.initiate_connection(App.OUTLOOK)
# ### background task save request.connectedAccountId to integrations table in supabase 
# ### then return the below redirect url to frontend
# print(f"Open this URL to authenticate: {request.redirectUrl}")


[2025-01-27 20:08:49,642][INFO] Logging is set to INFO, use `logging_level` argument or `COMPOSIO_LOGGING_LEVEL` change this


In [25]:
openai_client = OpenAI()

today = datetime.now().strftime("%Y-%m-%d")
task = f"get calendar slots for next week, today is {today}"

# Update the tools to include search capability
tools = toolset.get_tools(actions=[
    Action.OUTLOOK_OUTLOOK_UPDATE_CALENDAR_EVENT])


# First, let's search for your database
search_response = openai_client.chat.completions.create(
    model="gpt-4o",
    tools=tools,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Search for a database called DevOps DB in my Notion workspace"},
    ],
)
search_result = toolset.handle_tool_calls(search_response)
search_result


[]

### Shopify

In [ ]:
"""Auth"""
""" shopify not working :("""

from composio import ComposioToolSet, Composio, App
from dotenv import load_dotenv
load_dotenv()

client = Composio()
toolset = ComposioToolSet()
entity = toolset.get_entity()
request = entity.initiate_connection(App.SHOPIFY, connected_account_params={"shop": "fdi1gh-xg"})

print(f"Open this URL to authenticate: {request.redirectUrl}")


### Notion

In [5]:
"""Auth"""
from composio import ComposioToolSet, App

toolset = ComposioToolSet(entity_id="Jessica")
entity = toolset.get_entity()
request = entity.initiate_connection(App.NOTION)

print(f"Open this URL to authenticate: {request.redirectUrl}")


[2024-12-27 22:38:06,338][INFO] Logging is set to INFO, use `logging_level` argument or `COMPOSIO_LOGGING_LEVEL` change this


Open this URL to authenticate: https://api.notion.com/v1/oauth/authorize?client_id=17eb9b75-7ad0-4fc1-b01d-12bb09271484&redirect_uri=https%3A%2F%2Fbackend.composio.dev%2Fapi%2Fv1%2Fauth-apps%2Fadd&scope=&response_type=code&owner=user&state=production_eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJjb25uZWN0aW9uSWQiOiIxNDNkMGE5Yy1kNDUwLTRjZGQtODhmZi1jMDZmOTFlM2E2MzIiLCJpbnRlZ3JhdGlvbklkIjoiZjk3MjE3MzctYWNjOC00MmY5LWFlMTgtZjk5NzM3MGFiZjQ3IiwiYXBwTmFtZSI6Im5vdGlvbiIsImNsaWVudEluZm8iOnsicHJvamVjdElkIjoiMDEyNzE0NTktODYzZi00NzcwLWE4YTMtMTY2YWNiNzgxZjM3IiwibWVtYmVySWQiOiIyYmQyNjA1ZC0yYjk4LTQwZTgtYmM1Ni1mOGQ2NmExNTY5ZWMifSwiaWF0IjoxNzM1MzM5MDg3fQ.ku1n9q2ATg_71RbMC-OrBE_youknKQ1UPanTxaSPumQ&code_challenge=_qhTzmGt_gtb3uCjvqrs1lPPbriQ7VIQn6VnFe8mK10&code_challenge_method=S256


In [6]:
from composio_openai import ComposioToolSet, Action
from openai import OpenAI

openai_client = OpenAI()
composio_toolset = ComposioToolSet(entity_id="Jessica")

[2024-12-27 22:38:17,787][INFO] Logging is set to INFO, use `logging_level` argument or `COMPOSIO_LOGGING_LEVEL` change this


In [9]:
today = datetime.now().strftime("%Y-%m-%d")
task = "get the DB id for DevOps DB"

# Update the tools to include search capability
tools = composio_toolset.get_tools(actions=[
    Action.NOTION_SEARCH_NOTION_PAGE])

# First, let's search for your database
search_response = openai_client.chat.completions.create(
    model="gpt-4o",
    tools=tools,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Search for a database called DevOps DB in my Notion workspace"},
    ],
)
search_result = composio_toolset.handle_tool_calls(search_response)

db_id = search_result[0]['data']['response_data']['results'][0]['id']
print("DB ID:", db_id)

task = f"get the page to do's in DB with id {db_id}"

# Update the tools to include search capability
tools = composio_toolset.get_tools(actions=[
    Action.NOTION_QUERY_DATABASE])

# search_response = openai_client.chat.completions.create(
#     model="gpt-4o",
#     tools=tools,
#     messages=[
#         {"role": "system", "content": "You are a helpful assistant."},
#         {"role": "user", "content": "Search for a database called DevOps DB in my Notion workspace"},
#     ],
# )
# search_result = composio_toolset.handle_tool_calls(search_response)



[2024-12-27 22:39:20,262][INFO] Executing `NOTION_SEARCH_NOTION_PAGE` with params={'query': 'DevOps DB', 'filter_value': 'database', 'filter_property': 'object'} and metadata={} connected_account_id=None
[2024-12-27 22:39:23,270][INFO] Got response={'successfull': True, 'data': {'response_data': {'object': 'list', 'results': [{'object': 'database', 'id': '11453e39-0487-80f9-8b8c-de864568965a', 'cover': None, 'icon': None, 'created_time': '2024-10-03T20:18:00.000Z', 'created_by': {'object...


DB ID: 11453e39-0487-80f9-8b8c-de864568965a


### GMAIL

In [12]:

toolset = ComposioToolSet()
entity = toolset.get_entity(id="user_2mmXezcGmjZCf88gT2v2waCBsXv")

request = entity.initiate_connection(App.GMAIL)

print(f"Open this URL to authenticate: {request.redirectUrl}")


[2024-11-30 12:06:40,432][INFO] Logging is set to INFO, use `logging_level` argument or `COMPOSIO_LOGGING_LEVEL` change this


Open this URL to authenticate: https://accounts.google.com/o/oauth2/v2/auth?client_id=967786027051-8p515bt7ijf94c2bf4a5lev5jr6r7oc1.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fbackend.composio.dev%2Fapi%2Fv1%2Fauth-apps%2Fadd&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.modify&response_type=code&access_type=offline&prompt=consent&state=production_eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJjb25uZWN0aW9uSWQiOiJkYzM3ODg4Ni1mZDQxLTQ4NGUtYmMwMS1kZWI4MWQxNWVkZTMiLCJpbnRlZ3JhdGlvbklkIjoiZjYxNWYxYmMtZWJkZC00ZGNjLWEwZWMtNDdjZTU5YTNjMGJjIiwiYXBwTmFtZSI6ImdtYWlsIiwiY2xpZW50SW5mbyI6eyJwcm9qZWN0SWQiOiIwMTI3MTQ1OS04NjNmLTQ3NzAtYThhMy0xNjZhY2I3ODFmMzciLCJtZW1iZXJJZCI6IjJiZDI2MDVkLTJiOTgtNDBlOC1iYzU2LWY4ZDY2YTE1NjllYyJ9LCJpYXQiOjE3MzI5Njg0MDF9.qBRqYHWJobmhM6cli4vnIYMs4rKvrXml9e99qReLWVI&code_challenge=csn_1ubX0LSJuVrGZPFaQ9s9Ez5IzFs765c2cbyNnL4&code_challenge_method=S256


In [13]:
from composio_openai import ComposioToolSet, Action
from composio import Composio
from openai import OpenAI
from datetime import datetime 

from dotenv import load_dotenv
load_dotenv()

client = Composio()
openai_client = OpenAI()
composio_toolset = ComposioToolSet(entity_id="user_2mmXezcGmjZCf88gT2v2waCBsXv")


tools = composio_toolset.get_tools(actions=[Action.GMAIL_FETCH_EMAILS])
today = datetime.now().strftime("%Y-%m-%d")

task = f"fetch emails for this month of november 2024 for michael@flowon.ai. Today's date is {today}."

response = openai_client.chat.completions.create(
    model="gpt-4o",
    tools=tools,
    messages=[
        {"role": "system", "content": f"You are a helpful assistant. Today's date is {today}."},
        {"role": "user", "content": task},
    ],
)

result = composio_toolset.handle_tool_calls(response)

result

[2024-11-30 12:06:52,080][INFO] Logging is set to INFO, use `logging_level` argument or `COMPOSIO_LOGGING_LEVEL` change this
[2024-11-30 12:06:54,338][INFO] Executing `GMAIL_FETCH_EMAILS` with params={'user_id': 'michael@flowon.ai', 'query': 'after:2024/10/31 before:2024/12/01'} and metadata={} connected_account_id=None
[2024-11-30 12:06:56,605][INFO] Got response={'successfull': True, 'data': {'response_data': {'nextPageToken': '17877557403714355457', 'resultSizeEstimate': 201, 'messages': [{'threadId': '1937c66fd468ae77', 'messageId': '1937c66fd468ae77', 'messageTimestamp': '2024-11-30T09:29:03Z', 'la...


[{'successfull': True,
  'data': {'response_data': {'nextPageToken': '17877557403714355457',
    'resultSizeEstimate': 201,
    'messages': [{'threadId': '1937c66fd468ae77',
      'messageId': '1937c66fd468ae77',
      'messageTimestamp': '2024-11-30T09:29:03Z',
      'labelIds': ['CATEGORY_PROMOTIONS', 'UNREAD', 'INBOX'],
      'preview': {'subject': 'Winter Sun discounts ✅',
       'body': '20% OFF Flights! View online FLIGHTS CAR HIRE HOTELS GIFT CARDS Ready to level up your winter? Discounted Winter Sun flights are available for this weekend only. Why freeze when you could be soaking up'},
      'messageText': '20% OFF Flights! \r\n\r\n \t\r\n\r\n https://t1.marketing.ryanairemail.com/r/?id=t01902918-c4e5-494f-9903-42df110740ab,5cb9c,15845&e=dXRtX3NvdXJjZT1jcm0mdXRtX21lZGl1bT1lbWFpbCZ1dG1fY2FtcGFpZ249ZW1haWxfUl9GTElHSFRUQUNUX0NXZW5kMzAxMTI0X2diX25vc2VnX25vYWJfbm9kaXZfbm9wX25vbHMmcDE9JTQwOEZ6ZVVJMnphcEg1UGJHSnJpdHhXbDUlMkY1V20xcFZVdFdWU1lMV0VvOWN0YmVSVSUyRmxQcFV6cTQ4ZDhpME1LUW0zYSUy

### Google Calendar

In [ ]:
from composio_openai import ComposioToolSet, Action
from composio import Composio
from openai import OpenAI
from datetime import datetime 

from dotenv import load_dotenv
load_dotenv()

client = Composio()
openai_client = OpenAI()
composio_toolset = ComposioToolSet(entity_id="user_2mmXezcGmjZCf88gT2v2waCBsXv")

GOOGLECALENDAR_CREATE_EVENT: 'Action'
GOOGLECALENDAR_DELETE_EVENT: 'Action'
GOOGLECALENDAR_DUPLICATE_CALENDAR: 'Action'
GOOGLECALENDAR_FIND_EVENT: 'Action'
GOOGLECALENDAR_FIND_FREE_SLOTS: 'Action'
GOOGLECALENDAR_GET_CALENDAR: 'Action'
GOOGLECALENDAR_GET_CURRENT_DATE_TIME: 'Action'
GOOGLECALENDAR_LIST_CALENDARS: 'Action'
GOOGLECALENDAR_PATCH_CALENDAR: 'Action'
GOOGLECALENDAR_QUICK_ADD: 'Action'
GOOGLECALENDAR_REMOVE_ATTENDEE: 'Action'
GOOGLECALENDAR_UPDATE_EVENT: 'Action'

# tools = composio_toolset.get_tools(actions=[Action.GOOGLECALENDAR_LIST_CALENDARS])
# task = "find free slots for next week"

# tools = composio_toolset.get_tools(actions=[Action.GOOGLECALENDAR_GET_CALENDAR])
# task = "fetch calendar for id michael@flowon.ai"

tools = composio_toolset.get_tools(actions=[Action.GOOGLECALENDAR_FIND_FREE_SLOTS])
today = datetime.now().strftime("%Y-%m-%d")

task = f"Search for free slots for the next 14 days for id michael@flowon.ai. Today's date is {today}."

response = openai_client.chat.completions.create(
    model="gpt-4o",
    tools=tools,
    messages=[
        {"role": "system", "content": f"You are a helpful assistant. Today's date is {today}."},
        {"role": "user", "content": task},
    ],
)

result = composio_toolset.handle_tool_calls(response)

In [ ]:
from services.composio import get_calendar_slots, find_free_slots

await get_calendar_slots("user_2mmXezcGmjZCf88gT2v2waCBsXv","outlook")



In [2]:

tools = composio_toolset.get_tools(actions=[Action.GOOGLECALENDAR_CREATE_EVENT])

task = "Checkup appointment on Friday, November 29th, 2024 at 2:30 PM'Create"

today = datetime.now().strftime("%Y-%m-%d")

response = openai_client.chat.completions.create(
    model="gpt-4-turbo-preview",
    tools=tools,
    messages=[
        {"role": "system", "content": f"You are a helpful assistant. Today's date is {today}."},
        {"role": "user", "content": task},
    ],
)
result = composio_toolset.handle_tool_calls(response)
print(result)


NameError: name 'composio_toolset' is not defined

In [4]:
from services.composio import book_appointment_composio
import logging

logger = logging.getLogger(__name__)


In [1]:
from services.composio import book_appointment_composio

results = await book_appointment_composio("Checkup appointment on Friday, November 29th, 2024 at 2:30 PM", "user_2mmXezcGmjZCf88gT2v2waCBsXv")
results

[2024-11-27 22:44:35,585][INFO] Logging is set to INFO, use `logging_level` argument or `COMPOSIO_LOGGING_LEVEL` change this


func book_appointment_composio: booking


[2024-11-27 22:44:39,408][INFO] Executing `GOOGLECALENDAR_CREATE_EVENT` with params={'summary': 'Checkup Appointment', 'start_datetime': '2024-11-29T14:30:00', 'event_duration': '1h', 'calendar_id': 'primary'} and metadata={} connected_account_id=None
[2024-11-27 22:44:41,139][INFO] Got response={'successfull': True, 'data': {'response_data': {'kind': 'calendar#event', 'etag': '"3465494961802000"', 'id': 'lf5si11qgu07tpucer7pihpqns', 'status': 'confirmed', 'htmlLink': 'https://www.google.com/calendar/event?eid=bGY1c2kxMXFndTA3dHB1Y2Vy...


[{'successfull': True,
  'data': {'response_data': {'kind': 'calendar#event',
    'etag': '"3465494961802000"',
    'id': 'lf5si11qgu07tpucer7pihpqns',
    'status': 'confirmed',
    'htmlLink': 'https://www.google.com/calendar/event?eid=bGY1c2kxMXFndTA3dHB1Y2VyN3BpaHBxbnMgbWljaGFlbEBmbG93b24uYWk',
    'created': '2024-11-27T22:44:40.000Z',
    'updated': '2024-11-27T22:44:40.901Z',
    'summary': 'Checkup Appointment',
    'creator': {'email': 'michael@flowon.ai', 'self': True},
    'organizer': {'email': 'michael@flowon.ai', 'self': True},
    'start': {'dateTime': '2024-11-29T14:30:00Z', 'timeZone': 'UTC'},
    'end': {'dateTime': '2024-11-29T15:30:00Z', 'timeZone': 'UTC'},
    'iCalUID': 'lf5si11qgu07tpucer7pihpqns@google.com',
    'sequence': 0,
    'reminders': {'useDefault': True},
    'eventType': 'default'}},
  'error': None}]

In [3]:
results

'Appointment booked successfully.'

### Monday

### Slack

In [13]:
from composio import ComposioToolSet, App
import os 
from dotenv import load_dotenv
load_dotenv()

toolset = ComposioToolSet()

trigger_schema = toolset.get_trigger("OUTLOOK_OUTLOOK_SENT_MESSAGE_TRIGGER")
print(trigger_schema.config)


[2025-01-27 18:53:24,928][INFO] Logging is set to INFO, use `logging_level` argument or `COMPOSIO_LOGGING_LEVEL` change this


properties={} title='OutlookTriggerConfig' type='object' required=None


In [15]:
entity = toolset.get_entity()
response = entity.enable_trigger(
    app=App.OUTLOOK,
    trigger_name="OUTLOOK_OUTLOOK_SENT_MESSAGE_TRIGGER",
    config={"owner": "composiohq", "repo": "composio"},
)
print(response)




{'status': 'success', 'triggerId': '1e58dc70-c8e2-4831-898d-45b53ffc9565', 'isNew': False}


In [ ]:
listener = toolset.create_trigger_listener()

@listener.callback(filters={"trigger_name": "OUTLOOK_OUTLOOK_SENT_MESSAGE_TRIGGER"})
def callback_function(event):
    print(event)

print("Listening")
listener.wait_forever()


In [23]:
import requests
from dotenv import load_dotenv
import os 
load_dotenv()

api_key = os.getenv("COMPOSIO_API_KEY")
connected_account_id = "4db28254-01c8-4127-b99f-6caa73d22a9f"

url = f"https://backend.composio.dev/api/v1/connectedAccounts/{connected_account_id}"

headers = {"x-api-key": api_key}

response = requests.request("GET", url, headers=headers)

print(response.text)

{"integrationId":"f70793ba-b2ea-47c2-8044-b4950c5d5e61","connectionParams":{"scope":"********","scopes":"********","client_id":"********","expires_in":"********","token_type":"********","redirectUrl":"https://login.microsoftonline.com/common/oauth2/v2.0/authorize?client_id=05a98fa5-dad5-44ea-ad93-d190ee828ee1&redirect_uri=https%3A%2F%2Fbackend.composio.dev%2Fapi%2Fv1%2Fauth-apps%2Fadd&scope=Calendars.Read+Calendars.ReadWrite+Contacts.ReadWrite+Mail.Read+Mail.ReadWrite+Mail.Send+offline_access+User.Read&response_type=code&response_mode=query&prompt=consent&state=production_eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJjb25uZWN0aW9uSWQiOiI0ZGIyODI1NC0wMWM4LTQxMjctYjk5Zi02Y2FhNzNkMjJhOWYiLCJpbnRlZ3JhdGlvbklkIjoiZjcwNzkzYmEtYjJlYS00N2MyLTgwNDQtYjQ5NTBjNWQ1ZTYxIiwiYXBwTmFtZSI6Im91dGxvb2siLCJjbGllbnRJbmZvIjp7ImNsaWVudElkIjoiMDEyNzE0NTktODYzZi00NzcwLWE4YTMtMTY2YWNiNzgxZjM3IiwibWVtYmVySWQiOiIyYmQyNjA1ZC0yYjk4LTQwZTgtYmM1Ni1mOGQ2NmExNTY5ZWMifSwiaWF0IjoxNzMyNDA5MTc3fQ.uqWAv3H8YBgWFBySpXZzbJIFmfQtUESjn